<a href="https://colab.research.google.com/github/organisciak/Scripting-Course/blob/master/labs/09-mongodb-json-lab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Lab: MongoDB Introduction

This week's lab is a mini-lab.

Reminder - save your work. Go to File > Save a Copy in Drive to ensure that you have your work saved.

## DataFrame Selection

This is here just for reference. Don't forget it!

**Selecting rows by numeric index**

Provide `x:y` notation in : `df[10:14]`

**Selecting rows by index name**

Provide the name to `.loc[]`: `df.loc['Sherlock Holmes']`

**Selecting rows by inclusion criteria**

Provide any collection (e.g. a list or Series) of True/False values:

```
df[[True, False, False, True, True]]
```

```
df[df.year > 1996]
```
    
**Selecting multiple columns**

Provide a collection of strings, referencing the column names:

```
df[['genres', 'year']]
```
    
**Selecting single column (as Series)**

```
df['year']
```

Or:

```
df.year
```

Consider the latter as the shortcut, not the main way.

The output is a Series. To select a single column as a DataFrame, use list with only one value.

## Opening JSON

This code shows how you load JSON into Python. If you're not reading data from the internet, you don't need the `from smart_open` line.

In [ ]:
import json
from smart_open import open
with open('https://raw.githubusercontent.com/organisciak/Scripting-Course/master/data/cooking.json') as f:
    data = json.load(f)

print("There are ", len(data), "items.")

There are  39774 items.


What's done here? 

You load a text file, and 'parse' it for Python to understand that it is formatted as JSON.

`with open(...) as f` syntax opens a file, setting it to a variable named `f`. That can read the raw text from the file, but the `json.load` function parses the text (one big string!) to a Python object.

This particular data is a big list of foods: try printing the first few items:

In [ ]:
data[:2]

[{'cuisine': 'greek',
  'id': 10259,
  'ingredients': ['romaine lettuce',
   'black olives',
   'grape tomatoes',
   'garlic',
   'pepper',
   'purple onion',
   'seasoning',
   'garbanzo beans',
   'feta cheese crumbles']},
 {'cuisine': 'southern_us',
  'id': 25693,
  'ingredients': ['plain flour',
   'ground pepper',
   'salt',
   'tomatoes',
   'ground black pepper',
   'thyme',
   'eggs',
   'green tomatoes',
   'yellow corn meal',
   'milk',
   'vegetable oil']}]

### Questions

Q1) What type of cuisine is the 300th item (index=299) in the dataset? (*3pts*)

In [ ]:
q1_answer = "" #@param {type:"string"}

Q2) What food is the tenth ingredient of the 200th item in the data? Tip: you can count by hand for the answer, but see if you can grab it with code. (*3pts*)

In [ ]:
q2_answer = "" #@param {type:"string"}

Q3) If you had run `data2 = f.read()` instead of `data = json.load(f)`, what type of information would the `data2` variable hold? How or why is it different? (Tip: try running your code from Q1 on data2 to see what it looks like, or use the `type()` function to figure our the type of variable that `data2` is compared to `data`.) (*3pts*)

In [ ]:
q3_answer = "" #@param {type:"string"}


Q4) What's the difference between running `print(data2[300:500])`, and having the notebook auto-print by running a cell with `data2[300:500]` on the last line? (*3pts*)

In [ ]:
q4_answer = "" #@param {type:"string"}

## Setting up MongoDB

MongoDB is a separate database. If you work with Jupyter on your own computer, you have to install it. For this class, since we're using Colab online, we'll also use an online service that already has MongoDB installed.

For this class, *Dr. Organisciak has created a read-only database that you can work with*, so you don't have to create your own database.

*If* you would like to create your own online-hosted database *anyway*, you can do so with the following steps:
- 1. Create an account with [MongoDB Atlas](https://www.mongodb.com/cloud/atlas), by clicking the Try Free button.
- 2. Create a shared cluster in a free tier.
- 3. Click 'Connect', then create a user, a password, and add `0.0.0.0` to the whitelist. (This whitelist setting means people can connect from anywhere if they have your username and password, so it's okay for a learning environment but less secure for critical applications).
- 4. For 'Choose a Connection Method', select 'connect your application', then 'Python', then copy the connection string. This is the location to your server.

In [ ]:
#@title Connect to a MongoDB database
#@markdown This cell connects to a remote MongoDB instance.
#@markdown When it asks your password, paste in the password posted to Canvas.
!pip install dnspython
from urllib.parse import quote_plus
from pymongo import MongoClient
import os
from getpass import getpass

if os.path.exists('credentials.txt'):
    # Allow loading credentials for user, pw, url, one per line
    with open('credentials.txt', mode='r') as f:
        user, mongopw, cluster_url = [l.strip() for l in f.readlines()]
else:
    user = "scriptingStudent" #@param {type:"string"}
    cluster_url = "cluster0-ga5s0.mongodb.net" #@param {type:"string"}
    mongopw = getpass('Enter your MongoDB password for "{}":\n'.format(user))
    with open('credentials.txt', mode='w') as f:
        f.write("{}\n{}\n{}".format(user, mongopw, cluster_url))

client = MongoClient("mongodb+srv://{}:{}@{}/test?retryWrites=true&w=majority".format(quote_plus(user), quote_plus(mongopw), cluster_url))
db = client.week9

     |████████████████████████████████| 245kB 5.1MB/s 
Enter your MongoDB password for "scriptingStudent":
··········


The database that we're using is called 'week9', and has been set to a variable called `db`:

In [ ]:
db

Database(MongoClient(host=['cluster0-shard-00-00-ga5s0.mongodb.net:27017', 'cluster0-shard-00-02-ga5s0.mongodb.net:27017', 'cluster0-shard-00-01-ga5s0.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, retrywrites=True, w='majority', authsource='admin', replicaset='Cluster0-shard-0', ssl=True), 'week9')

### Connecting to a Collection

Here, I connect to the 'cooking' collection of `db`. If it doesn't exist, it will be created. (The account you're connecting is read-only, though, so you won't be able to create new collections in Dr. O's database!)

In [ ]:
collection = db.cooking

### Inserting Data

Here's how you insert the `data` from before. It's a list - data[0] is one 'document', data[1] is another one, and so on - so we can use `insert_many`:

```
collection.insert_many(data)
```

**The data is already on our class database**, so you don't have to insert it.

To see a count of how many records we have, run:

In [ ]:
collection.estimated_document_count()

It's also possible to use `insert_one` for a single record:

```collection.insert_one(your_record)```

(Again, we're connected to our class database read-only, so you can't insert here unless you create your own DB).

# Questions

Q5) How many documents are in the cooking collection? (*3pts*)
   - 7954
   - 39774
   - 79548

In [ ]:
q5_answer = "" #@param ["", "7954", "39774", "79548"]

Q6) Match the cuisine to the number of documents that are that type of cuisine. Tip: You can run `count()` on the result of `find()` (*5pts*).
   - Cuisines: a) 'japanese', b) 'mexican', c) 'italian'
   - Counts: 6438, 7838, 1423

In [ ]:
q6a_answer = "" #@param ["", "6438", "7838", "1423"]

In [ ]:
q6b_answer = "" #@param ["", "6438", "7838", "1423"]

In [ ]:
q6c_answer = "" #@param ["", "6438", "7838", "1423"]

Q7) How many results have liver as an ingredient? Tip: If you want to inspect the results to doublecheck but worry about printing a BFD (a 'big dataset'), you can try `find_one` instead of `find`. Remember that for the answer itself, you want `find` and `count` though! (*4pts*)

In [ ]:
q7_answer = "" #@param {type:"string"}

`$in` and `$all` questions:

Q8) How many results have both 'duck' and 'chinese five-spice powder'? (*3pts*)

In [ ]:
q8_answer = "" #@param {type:"string"}


Q9) How many results have either 'duck' or 'chinese five-spice powder'? (*3pts*)

In [ ]:
q9_answer = "" #@param {type:"string"}

# Submission Instructions

In [ ]:
#@markdown ### First, Enter your name for grading
my_name = "" #@param { type:'string' }

#@markdown _Have you saved your work for yourself? Don't forget to Save a Copy in Drive so that you have your progress._

In [ ]:
#@markdown ### Second, check your work:

#@markdown - have you answered all the questions?
#@markdown     - Some answers can be checked automatically - just run this cell.
#@markdown - Does this notebook run from top to bottom?
#@markdown     - Go to "Runtime > Restart and run all..." to check. Do all the cells run, to the very bottom, or is there a cell in the middle with an error?
#@markdown - Have you completed all the answers where you entered code, keeping the `# Answer-Qx` line at the start of those cells?

#@markdown *A lab that the professor has to fix manually will lose 10pts - run the checks!*

lab1 = dict(
    q1 = dict(entrytype='cell', pts=5),
    q2 = dict(entrytype='var', pts=5),
    q3 = dict(entrytype='cell', pts=5),
    q4a = dict(entrytype='var', pts=5),
    q4b = dict(entrytype='var', pts=5),
    q4c = dict(entrytype='var', pts=10),
    q5 = dict(entrytype='var', pts=5),
    q6 = dict(entrytype='cell', pts=5),
    q7 = dict(entrytype='cell', pts=5),
    q8a = dict(entrytype='var', pts=4),
    q8b = dict(entrytype='var', pts=4),
    q8c = dict(entrytype='var', pts=4),
    q9a = dict(entrytype='var', pts=4),
    q9b = dict(entrytype='var', pts=4),
    q10 = dict(entrytype='var', pts=8),
    q11 = dict(entrytype='var', pts=7),
    q12 = dict(entrytype='var', pts=5),
    q13 = dict(entrytype='cell', pts=10),
)
var_qs = [k for k,v in lab1.items() if v['entrytype'] == 'var']
for q in var_qs:
  qvar = '{}_answer'.format(q) 
assert(qvar) in globals(), "I don't see '{}' - did you run the cell where you ran it?".format(qvar)
answer = globals()[qvar]
assert answer != "", "{} is blank - did you mean to do that?".format(qvar)
print("Good work. The ones that I can pre-check look correct.")

#@markdown ### Finally, submit it

#@markdown - Download the file with "File > Download .ipynb" and submit it to the Canvas assignment page.